# **StreamSmart: Personalized Movie Recommendations**

## 🚦 Overview
1. Business Understanding
2. Data Understanding / EDA
3. Data Preparation & Splits
4. Baselines & KNN CF
5. SVD / ALS & Hybrid
6. Evaluation, (Optional) Ensemble Re-ranker, Conclusion

---

## 🎯 **Business Understanding**

Intro, stakeholder, long-tail problem, success metric

---

## 🔎 **Data Understanding / EDA**

Dataset overview, plots: ratings distribution, user activity, movie popularity

---

## 🧪 **Data Preparation & Splits**

Chronological per-user split, avoid leakage

---

## 🧱 **Baselines & KNN CF**

Popularity, user mean, item mean, user-KNN, item-KNN

---

## 🔬 **SVD / ALS & Hybrid**

SVD, ALS, content-based fallback hybrid

---

## 📊 **Evaluation, Ensemble Re-ranker, & Conclusion**

Compare models, metrics table, stakeholder implications

---